# 형태소 분석 및 TDM 만들기

In [1]:
import pandas as pd

In [2]:
review = pd.read_excel('review.xlsx', index_col=0)

In [3]:
review

,review,label
0,"군대에서 체단실꾸준히 가고, 단백질 챙겨먹으려 샀는데 5kg짜리 700g정도밖에 ...",0.0
1,리뷰 많아서 아이스라떼로 사봤는데 맛있네요 담에도 이거 살듯,0.0
2,I've been looking for a whey protein with goo...,0.0
3,왜 사람들이 추천하는 이유를 알겠네요 달달하니 운동전에 한스쿱 운동끝나고 한스쿱 ...,0.0
4,솔티드 카라멜이라고 되어있지만 맛은 물 150ml에 태워 먹었을때 율무차 맛이 나...,0.0
...,...,...
4919,처음에 열나는거 맞는데 뭔가 기대했던 것 만큼 엄청나지는 않음. 더 먹어봐야 알것...,1.0
4920,배송이 한달걸리는건 코로나 때문에 그렇다쳐도 ㅋㅋㅋ먹을수 잇게 배달은 해와야지 장...,1.0
4921,타회사 보충제 4년정도먹다가 이번에 마이프로틴 으로 주문했는데 3주가 다되가는데 ...,1.0
4922,초과용량 먹고 뒤지라는 건가요. ?,1.0


In [1]:
#!pip install kiwipiepy

In [7]:
from kiwipiepy import Kiwi

In [8]:
kiwi = Kiwi()

In [9]:
kiwi.prepare()

0

In [14]:
def extract_noun(text):
    nouns = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if pos.startswith('N'):
            nouns.append(token)
    return nouns

# TDM with(kiwi)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv = CountVectorizer(max_features=500, tokenizer=extract_noun)

In [17]:
tdm_kiwi = cv.fit_transform(review['review'])

In [18]:
cv.get_feature_names()

['1스쿱',
 '2스쿱',
 '가격',
 '가능',
 '가루',
 '가성',
 '가성비',
 '가슴',
 '가입',
 '가지',
 '간',
 '감',
 '감미료',
 '감사',
 '갑',
 '값',
 '강',
 '강추',
 '강추합니',
 '개',
 '개선',
 '개월',
 '개인',
 '걍',
 '거',
 '거기',
 '거부감',
 '거품',
 '걱정',
 '건강',
 '것',
 '게',
 '결과',
 '경우',
 '계속',
 '계열',
 '계획',
 '고민',
 '골드',
 '골든시럽',
 '곳',
 '구매',
 '구입',
 '국내',
 '굿',
 '그거',
 '그것',
 '그린티',
 '근',
 '근육',
 '근육통',
 '글루타민',
 '기간',
 '기대',
 '기본',
 '기분',
 '기존',
 '기준',
 '꺼',
 '꿀',
 '꿀맛',
 '끝',
 '나',
 '나름',
 '날',
 '남',
 '내',
 '내추럴',
 '내츄',
 '냄새',
 '넛',
 '네추',
 '네츄',
 '년',
 '녹차',
 '눈',
 '느낌',
 '다양',
 '다음',
 '다음번',
 '다이어트',
 '다행',
 '단맛',
 '단백',
 '단백질',
 '단점',
 '달',
 '달달',
 '닭',
 '담',
 '당',
 '대',
 '대란',
 '대박',
 '대부분',
 '대비',
 '대신',
 '대용',
 '덩어리',
 '데',
 '도움',
 '도전',
 '도착',
 '돈',
 '동안',
 '두유',
 '둘',
 '뒤',
 '드',
 '득근',
 '듯',
 '등',
 '딸기',
 '딸기우유',
 '땅콩',
 '때',
 '때문',
 '떼',
 '라무네',
 '라즈베리',
 '량',
 '럴',
 '레몬',
 '리',
 '리뷰',
 '마',
 '마음',
 '마이',
 '마이프로',
 '마이프로틴',
 '마지막',
 '마차',
 '마프',
 '만',
 '만족',
 '만큼',
 '말',
 '말차',
 '말차라떼',
 '맘',
 '맛',
 '맛나',
 '망고',
 '메이플',
 '면

In [20]:
word_count_kiwi = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm_kiwi.sum(axis=0).flat
})

In [21]:
word_count_kiwi.sort_values('빈도', ascending = False)

,단어,빈도
140,맛,6160
24,거,1357
30,것,1089
41,구매,946
158,물,935
...,...,...
279,어디,18
91,대,18
388,진심,18
237,순위,17


# 불용어 사전

In [52]:
# 형태소 분석 결과를 토대로 이해가 안가거나 쓸모없는 단어들을 제거
stop_words = ['프로틴','구매','것','제품','생각','주문','보충제','처음','5','이','수','저','이거','1','분','아이솔레이트','2','웨이','때','~','3','거',
              '일','개인','이것','kg','에','중','후','한','처럼','10','만','ml','키','번','크레아틴','2.5','7',]

# 불용어 사전을 사용한 형태소 분석 - kiwi

In [60]:
def extract_noun(text):
    nouns = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if pos.startswith('N'):
            nouns.append(token)
    return nouns

In [61]:
cv = CountVectorizer(max_features=500, stop_words=stop_words, tokenizer=extract_noun)

In [62]:
tdm_kiwi = cv.fit_transform(review['review'])

C:\Users\82107\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['보충', '아이솔레', '처', '티', '프로'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [63]:
cv.get_feature_names()

['1스쿱',
 '2스쿱',
 '가격',
 '가능',
 '가루',
 '가성',
 '가성비',
 '가슴',
 '가입',
 '가지',
 '간',
 '감',
 '감미료',
 '감사',
 '감안',
 '갑',
 '값',
 '강',
 '강추',
 '강추합니',
 '개',
 '개선',
 '개월',
 '걍',
 '거기',
 '거부감',
 '거품',
 '걱정',
 '건강',
 '검색',
 '게',
 '결과',
 '경우',
 '계속',
 '계열',
 '계획',
 '고민',
 '골드',
 '골든시럽',
 '곳',
 '구입',
 '국내',
 '굿',
 '그거',
 '그것',
 '그린티',
 '근',
 '근육',
 '근육통',
 '글루타민',
 '금액',
 '기간',
 '기대',
 '기본',
 '기분',
 '기존',
 '기준',
 '꺼',
 '꿀',
 '꿀맛',
 '끝',
 '나',
 '나름',
 '날',
 '남',
 '내',
 '내추럴',
 '내츄',
 '냄새',
 '넛',
 '네추',
 '네츄',
 '년',
 '녹차',
 '녹차라떼',
 '눈',
 '느낌',
 '다',
 '다양',
 '다음',
 '다음번',
 '다이어트',
 '다행',
 '단맛',
 '단백',
 '단백질',
 '단점',
 '달',
 '달달',
 '닭',
 '담',
 '당',
 '대',
 '대란',
 '대박',
 '대부분',
 '대비',
 '대신',
 '대용',
 '덩어리',
 '데',
 '도움',
 '도전',
 '도착',
 '돈',
 '동안',
 '두유',
 '둘',
 '뒤',
 '드',
 '득근',
 '듯',
 '등',
 '딸기',
 '딸기우유',
 '땅콩',
 '때문',
 '떼',
 '라떼',
 '라무네',
 '라즈베리',
 '량',
 '럴',
 '레몬',
 '리',
 '리뷰',
 '마',
 '마음',
 '마이',
 '마이프로',
 '마이프로틴',
 '마지막',
 '마차',
 '마프',
 '만족',
 '만큼',
 '말',
 '말차',
 '말차라떼',
 '맘',
 '맛',
 '맛나',
 '망고',
 '맹물',

In [65]:
word_count_kiwi = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm_kiwi.sum(axis=0).flat
})

In [66]:
word_count_kiwi.sort_values('빈도', ascending = False)

,단어,빈도
140,맛,6160
161,물,935
304,우유,928
306,운동,899
447,티,691
...,...,...
29,검색,16
464,피로,16
143,맹물,16
144,먹어봣,16


In [68]:
#word_count_kiwi.to_excel('word_count_kiwi.xlsx') 데이터 저장